<a href="https://colab.research.google.com/github/yerimmms/calibrate-moe/blob/calibrate-moe/humalevalpack_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bigcode-project/bigcode-evaluation-harness.git
%cd bigcode-evaluation-harness

Cloning into 'bigcode-evaluation-harness'...
remote: Enumerating objects: 4282, done.
remote: Counting objects: 100% (1759/1759), done.
remote: Compressing objects: 100% (561/561), done.
remote: Total 4282 (delta 1380), reused 1424 (delta 1189), pack-reused 2523
Receiving objects: 100% (4282/4282), 932.66 KiB | 4.36 MiB/s, done.
Resolving deltas: 100% (2919/2919), done.
/content/bigcode-evaluation-harness


In [2]:
!python -V

Python 3.10.12


In [ ]:
!pip install -e .

In [ ]:
# for TF-TRT Warning: Could not find TensorRT
# !pip install tensorflow-gpu==2.8.0

In [11]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [17]:
!huggingface-cli env


Copy-and-paste the text below in your GitHub issue.

- huggingface_hub version: 0.23.2
- Platform: Linux-6.1.85+-x86_64-with-glibc2.35
- Python version: 3.10.12
- Running in iPython ?: No
- Running in notebook ?: No
- Running in Google Colab ?: No
- Token path ?: /root/.cache/huggingface/token
- Has saved token ?: True
- Who am I ?: yrju
- Configured git credential helpers: 
- FastAI: 2.7.15
- Tensorflow: 2.15.0
- Torch: 2.3.0+cu121
- Jinja2: 3.1.4
- Graphviz: 0.20.3
- keras: 2.8.0
- Pydot: 1.4.2
- Pillow: 9.4.0
- hf_transfer: N/A
- gradio: N/A
- tensorboard: N/A
- numpy: 1.25.2
- pydantic: 2.7.3
- aiohttp: 3.9.5
- ENDPOINT: https://huggingface.co
- HF_HUB_CACHE: /root/.cache/huggingface/hub
- HF_ASSETS_CACHE: /root/.cache/huggingface/assets
- HF_TOKEN_PATH: /root/.cache/huggingface/token
- HF_HUB_OFFLINE: False
- HF_HUB_DISABLE_TELEMETRY: False
- HF_HUB_DISABLE_PROGRESS_BARS: None
- HF_HUB_DISABLE_SYMLINKS_WARNING: False
- HF_HUB_DISABLE_EXPERIMENTAL_WARNING: False
- HF_HUB_DISABLE_I

In [27]:
### 환경변수 설정 ###
MODEL_NAME="yrju/CodeLlaMa-7B-dare-ties"
MAX_LENGTH=512
PROMPT="instruct"
TASK_NAME="humanevalfixtests-python"
PRECISION="fp16"

# MODEL_NAME="TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# PRECISION="bf16"

In [ ]:
# HumanEvalFix: In this task models are provided with a solution with a subtle bug and several unit tests. The task is to fix the function.
# There is a variant of this task where the function docstring instead of the unit tests are provided,
# which can be selected via humanevalfixdocs.
!accelerate launch main.py \
  --model {MODEL_NAME} \
  --max_length_generation {MAX_LENGTH} \
  --prompt {PROMPT} \
  --tasks humanevalfixtests-python \
  --temperature 0.2 \
  --n_samples 20 \
  --batch_size 10 \
  --precision {PRECISION} \
  --allow_code_execution \
  --save_generations

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-06-06 14:11:32.996423: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2024-06-06 14:11:32.996459: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Selected Tasks: ['humanevalfixtests-python']
Loading model in bf16
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for bigcode/humanevalpack contain

In [ ]:
# HumanEvalExplain: In this task models need to explain a HumanEval solution (without docstring)
# and subsequently regenerate the solution given only the model's own explanation.
# Thus, it requires two runs.
# The first one generates the descriptions, the second loads the descriptions, generates the solution & is scored.

!accelerate launch main.py \
  --model {MODEL_NAME} \
  --max_length_generation {MAX_LENGTH} \
  --prompt {PROMPT} \
  --tasks humanevalexplaindescribe-python \
  --temperature 0.2 \
  --n_samples 20 \
  --batch_size 10 \
  --generation_only \
  --save_generations

!accelerate launch main.py \
  --model {MODEL_NAME} \
  --max_length_generation {MAX_LENGTH} \
  --prompt {PROMPT} \
  --load_data_path generations_humanevalexplaindescribe-python.json \
  --tasks humanevalexplainsynthesize-python \
  --temperature 0.2 \
  --n_samples 1 \
  --batch_size 1 \
  --allow_code_execution

In [ ]:
# HumanEvalSynthesize: This is like HumanEval but with human translations for JavaScript, Java, Go, C++ and Rust.
# It is based on HumanEval-X, however, with additional fixes and improvements documented here.
!accelerate launch main.py \
  --model {MODEL_NAME} \
  --max_length_generation {MAX_LENGTH} \
  --prompt {PROMPT} \
  --tasks humanevalsynthesize-python \
  --temperature 0.2 \
  --n_samples 20 \
  --batch_size 10 \
  --allow_code_execution \
  --save_generations

In [ ]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("yrju/CodeLlaMa-7B-dare-ties")
# model = AutoModelForCausalLM.from_pretrained("yrju/CodeLlaMa-7B-dare-ties")